# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Hardware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
versioninfo()

In [ ]:
using Pkg

In [ ]:
Pkg.add("Revise")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")
Pkg.add(url="https://github.com/bwbioinfo/KEGGAPI.jl")
Pkg.add("TestImages")
Pkg.add("Images")
Pkg.add("FileIO")
Pkg.add("Colors")

In [ ]:
using Revise
using DataFrames
using BenchmarkTools
using KEGGAPI
using Images
using TestImages
using Colors

# Case 4: Target molecules information at KEGG



### 1. Get gene id from KEGG Database

To determine if a molecule is included in KEGG database and identify the encoding gene in the species of interest, the function find takes as input the string "genes" and the name of the target molecule

In [ ]:
@time kegg_genes = KEGGAPI.find("genes", "CD19")
DataFrame(
  kegg_genes.data,
  kegg_genes.colnames
)

#### 1.1 Get gene information

In [ ]:
@time kegg_genes_info = KEGGAPI.kegg_get(["hsa:930"])
split(kegg_genes_info[2][1], "\n")

#### 1.2 Get KEGG ortholog group

In [ ]:
@time kegg_ko = KEGGAPI.link("ko", "hsa:930")
DataFrame(
  kegg_ko.data,
  kegg_ko.colnames
)

### 2. Pathway(s) including the molecule of interest

To obtain pathways associated to a gene the input of the "link" function are the string "pathway", and KEGG gene identifier.

In [ ]:
@time kegg_pathway = KEGGAPI.link("pathway", "hsa:930")
DataFrame(
    kegg_pathway.data,
    kegg_pathway.colnames
)

### 3. Drug(s) associated with molecule of interest

To obtain the drugs associated to a molecule the input of the "link" function is the string "drug", and KEGG gene identifier.

In [ ]:
@time kegg_drug= KEGGAPI.link("drug", "hsa:930")
DataFrame(
    kegg_drug.data,
    kegg_drug.colnames
)

### 4. Get drug(s) information from Kegg Database

To get information on the identified drugs included in KEGG database, the function kegg_get takes as input an array of drug KEGG identifier as "dr:DXXXXX

In [ ]:
@time kegg_drug_info = KEGGAPI.kegg_get(kegg_drug.data[2]);
split(kegg_drug_info[2][1], "\n")

### 5. Get image of the pathway of interest

The get_image function is to download a any image, the imput is the compound number as "path:XXXXXXXX"

The save_image function is to save the figure in a png file. The input is a string with the name of the file and the extension ".png"

In [ ]:
@time kegg_pathway_image = KEGGAPI.get_image("path:hsa04151")
@time KEGGAPI.save_image(kegg_pathway_image, "pathway.png")

### 6. Visualize download compound

In [ ]:
img = load("pathway.png")